In [1]:
#import all the required Libraries 
import pandas as pd  
import numpy as np
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
import string
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import itertools
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')

print('importing done')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
importing done


In [2]:
#Read the training dataset 
pharma_train=pd.read_csv('/content/drive/My Drive/Python/ZS_original/train.csv')
pharma_train

,job_no,job_description,job_type,category
0,Id-12765,Zest Scientific is searching for an accomplis...,Permanent,"Pharmaceutical, Healthcare and Medical Sales"
1,Id-22925,"En el mundo de los CRO's típicos, esta compañí...",Permanent,Clinical Research
2,Id-1321,Asha Mistry of Umbilical Life is recruiting an...,Contract/Interim,Pharmaceutical Marketing
3,Id-9473,Sales Representative - Laser in der Medizinte...,Permanent,"Pharmaceutical, Healthcare and Medical Sales"
4,Id-14952,Field Service Engineer - Life Science\r\r\r\nC...,Permanent,Manufacturing & Operations
...,...,...,...,...
17652,Id-1708,Evolve Selection are currently recruiting for ...,Permanent,Manufacturing & Operations
17653,Id-18516,Therapy Area: Respiratory\r\r\r\nPackage: Nego...,Permanent,Science
17654,Id-13531,COMPANY\r\r\r\nA specialised health economics ...,Permanent,Medical Information and Pharmacovigilance
17655,Id-21805,Asha Mistry de Umbilical Life está reclutando ...,Permanent,Clinical Research


In [3]:
#read the testing dataset
pharma_test=pd.read_csv('/content/drive/My Drive/Python/ZS_original/test.csv')
pharma_test

,job_no,job_description
0,Id-1,SAS Statistical Programmer II. UK. SDTM. ADaM....
1,Id-2,Therapy Area: Respiratory\r\r\r\nPackage: Nego...
2,Id-4,RBW consulting are working in partnership with...
3,Id-6,One of my long term clients is currently look...
4,Id-7,Unique package - Statistical Programmer - CRO\...
...,...,...
11766,Id-29421,Unternehmen:\r\r\r\n Grafschaft\r\r\r\nPositio...
11767,Id-29422,Austin Fraser freut sich auf die Zusammenarbei...
11768,Id-29425,Hyper Recruitment Solutions ist derzeit auf de...
11769,Id-29426,Leiter des globalen PV - Compliance & Training...


In [4]:
pharma_sample=pd.read_csv('/content/drive/My Drive/Python/ZS_original/sample_submission.csv')
pharma_sample

,job_no,category,job_type
0,Id-1,Data Management and Statistics,Permanent
1,Id-2,"Pharmaceutical, Healthcare and Medical Sales",Permanent
2,Id-4,Science,Permanent
3,Id-6,Pharmaceutical Marketing,Permanent
4,Id-7,Clinical Research,Permanent


In [5]:
#dropping the duplicates and nan values
pharma_train.drop_duplicates(inplace=True)
pharma_train.dropna(axis=0,inplace=True)
pharma_test.drop_duplicates(inplace=True)
pharma_test.dropna(axis=0,inplace=True)


In [6]:
#printing the information of training data 
pharma_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17657 entries, 0 to 17656
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_no           17657 non-null  object
 1   job_description  17657 non-null  object
 2   job_type         17657 non-null  object
 3   category         17657 non-null  object
dtypes: object(4)
memory usage: 689.7+ KB


In [7]:
#counting the no. of each value occured category in trainning dataset
pharma_train.category.value_counts()

Pharmaceutical, Healthcare and Medical Sales    4507
Clinical Research                               3172
Science                                         2932
Manufacturing & Operations                      2235
Regulatory Affairs                              1163
Pharmaceutical Marketing                        1071
Data Management and Statistics                   829
Quality-assurance                                703
Medical Information and Pharmacovigilance        561
Medical Affairs / Pharmaceutical Physician       458
Pharmacy                                          26
Name: category, dtype: int64

In [8]:
#counting the no. of each value occured job type in trainning dataset
pharma_train.job_type.value_counts()

Permanent             15777
Contract/Interim       1414
Contract/Temp           305
Temporary/Seasonal      113
Any                      25
Part-Time                23
Name: job_type, dtype: int64

In [9]:
pharma_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11771 entries, 0 to 11770
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_no           11771 non-null  object
 1   job_description  11771 non-null  object
dtypes: object(2)
memory usage: 275.9+ KB


In [10]:
pharma_train.describe()

,job_no,job_description,job_type,category
count,17657,17657,17657,17657
unique,17657,7006,6,11
top,Id-16239,Experienced Medical Writing - Germany - Consul...,Permanent,"Pharmaceutical, Healthcare and Medical Sales"
freq,1,38,15777,4507


In [11]:
pharma_test.describe()

,job_no,job_description
count,11771,11771
unique,11771,5669
top,Id-10823,RBW consulting are working in partnership with...
freq,1,28


In [12]:
pharma_train.job_description[2]

'Asha Mistry of Umbilical Life is recruiting an \r\r\r\nexperienced Account Manager\r\r\r\n for a medical communications agency based in Cheshire on a \r\r\r\nfreelance 6 month contract\r\r\r\n.\r\r\r\nThis is a great opportunity to work with a medical communications agency that works with clients in the pharmaceutical sector.\r\r\r\nHave you got considerable experience managing and developing client accounts in the pharmaceutical or healthcare sector? The ideal candidate will have significant account management experience developed within a medical communications agency.\r\r\r\nExperience and qualifications:\r\r\r\nA life-sciences degree is preferred, but must be educated to degree level\r\r\r\nMust have experience within a medical communications agency or healthcare advertising agency\r\r\r\nAt least 2 years’ Account Management experience is required in medical communications\r\r\r\nAble to develop client relationships and work closely with clients on budgets and deadlines\r\r\r\nMus

In [13]:
pharma_test.job_description[2]

'RBW consulting are working in partnership with a specialist Biometric consultancy who are looking for an experienced Principal Statistician to work remotely from the UK and join their expanding team. With a focus on providing high quality support across Biometrics, Regulatory Affairs and Quality Assurance this is a great time to join a company with a team of specialists who pride themselves on the quality of work and flexible approach to clients needs.\r\r\r\nThe company are looking for a Principal Statistician that enjoys working across a range of therapeutic areas and will have the opportunity to work on interesting and exciting projects for a number of pharmaceutical and biotech clients.\r\r\r\nMy client and I would like to hear from you if you have at least 5 years experience of Biostatistics in a clinical environment and a degree in a statistics/mathematics or computing-related discipline.\r\r\r\nPlease see below a selection of the responsibilities, should you wish to find out mo

In [14]:
#making copies of training and testing datasets
train=pharma_train.copy()
test=pharma_test.copy()

In [15]:
train.isnull().sum()

job_no             0
job_description    0
job_type           0
category           0
dtype: int64

In [16]:
#printing the information about predicting job type.
print('total number of job type', train.job_type.nunique())
print('total job type', train.job_type.unique())

total number of job type 6
total job type ['Permanent' 'Contract/Interim' 'Contract/Temp' 'Any' 'Part-Time'
 'Temporary/Seasonal']


In [17]:
#printing the information about predicting category.
print('total number of category', train.category.nunique())
print('total category', train.category.unique())

total number of category 11
total category ['Pharmaceutical, Healthcare and Medical Sales' 'Clinical Research'
 'Pharmaceutical Marketing' 'Manufacturing & Operations' 'Science'
 'Medical Affairs / Pharmaceutical Physician' 'Regulatory Affairs'
 'Medical Information and Pharmacovigilance'
 'Data Management and Statistics' 'Quality-assurance' 'Pharmacy']


In [18]:
#making the pedictive/dependent variable
y=train['job_type']
y

0               Permanent
1               Permanent
2        Contract/Interim
3               Permanent
4               Permanent
               ...       
17652           Permanent
17653           Permanent
17654           Permanent
17655           Permanent
17656           Permanent
Name: job_type, Length: 17657, dtype: object

In [19]:
#making dataset of independent variables
X=train.drop(['job_type', 'category'], axis=1)
X

,job_no,job_description
0,Id-12765,Zest Scientific is searching for an accomplis...
1,Id-22925,"En el mundo de los CRO's típicos, esta compañí..."
2,Id-1321,Asha Mistry of Umbilical Life is recruiting an...
3,Id-9473,Sales Representative - Laser in der Medizinte...
4,Id-14952,Field Service Engineer - Life Science\r\r\r\nC...
...,...,...
17652,Id-1708,Evolve Selection are currently recruiting for ...
17653,Id-18516,Therapy Area: Respiratory\r\r\r\nPackage: Nego...
17654,Id-13531,COMPANY\r\r\r\nA specialised health economics ...
17655,Id-21805,Asha Mistry de Umbilical Life está reclutando ...


In [20]:
#Label encoding the dependent variable for feeding into the model
enc=LabelEncoder()
y=enc.fit_transform(y)
y
z=y.copy()

In [21]:
 # function to get unique values 
def unique(list1): 
	x = np.array(list1) 
	print(np.unique(x)) 
	
 
unique(z)

[0 1 2 3 4 5]


In [22]:
test.reset_index(inplace=True)


In [23]:
#applying stemming and removing stopwords from the discription column of traing dataset
ps = PorterStemmer()
corpus = []
for i in range(0, len(train)):
    review = re.sub('[^a-zA-Z]', ' ', train['job_description'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [24]:
corpus[2]

'asha mistri umbil life recruit experienc account manag medic commun agenc base cheshir freelanc month contract great opportun work medic commun agenc work client pharmaceut sector got consider experi manag develop client account pharmaceut healthcar sector ideal candid signific account manag experi develop within medic commun agenc experi qualif life scienc degre prefer must educ degre level must experi within medic commun agenc healthcar advertis agenc least year account manag experi requir medic commun abl develop client relationship work close client budget deadlin must strong attent detail familiar varieti therapi area pleas note must elig work uk appli role find role pleas contact asha mistri umbil life email appli ref mistri fc'

In [25]:
#applying stemming and removing stopwords from the discription column of testing dataset
ps = PorterStemmer()
corpus1 = []
for i in range(0, len(test)):
    review = re.sub('[^a-zA-Z]', ' ', test['job_description'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus1.append(review)

In [26]:
corpus1[2]

'rbw consult work partnership specialist biometr consult look experienc princip statistician work remot uk join expand team focu provid high qualiti support across biometr regulatori affair qualiti assur great time join compani team specialist pride qualiti work flexibl approach client need compani look princip statistician enjoy work across rang therapeut area opportun work interest excit project number pharmaceut biotech client client would like hear least year experi biostatist clinic environ degre statist mathemat comput relat disciplin pleas see select respons wish find detail happi answer question may respons includ project manag act main point contact assign project provid strateg advic client matter concern statist need perform statist activ statist consult sap develop sa program produc efficaci safeti tabl list figur analys minimum requir msc phd statist relat field good clinic practic gcp intern confer harmon ich guidelin previou leadership experi team sub team profici use cd

In [27]:
test.shape

(11771, 3)

In [28]:
#Applying Tfidf Vectorizer to train dataset
tfidf_v=TfidfVectorizer(max_features=10000,ngram_range=(1,5))
X_train = tfidf_v.fit_transform(corpus).toarray()

In [29]:
X_train.shape

(17657, 10000)

In [30]:
#Applying tfidf Vectorizer to train dataset
tfidf_v1=TfidfVectorizer(max_features=10000,ngram_range=(1,5))
M_test = tfidf_v1.fit_transform(corpus1).toarray()

In [31]:
M_test.shape

(11771, 10000)

In [32]:
#Making a copy of dependent variable
y_train=y.copy()

In [33]:
#getting the feature names 
tfidf_v.get_feature_names()[:20]

['ab',
 'aber',
 'aber nicht',
 'abil',
 'abil adapt',
 'abil build',
 'abil commun',
 'abil develop',
 'abil drive',
 'abil drive requir',
 'abil drive requir role',
 'abil drive requir role would',
 'abil grow',
 'abil grow higher',
 'abil grow higher role',
 'abil grow higher role futur',
 'abil lead',
 'abil manag',
 'abil travel',
 'abil understand']

In [34]:
#getting the params 
tfidf_v.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': 10000,
 'min_df': 1,
 'ngram_range': (1, 5),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': None,
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'use_idf': True,
 'vocabulary': None}

In [35]:
#making the dataframe from the vectorized data
count_df = pd.DataFrame(X_train, columns=tfidf_v.get_feature_names())
count_df.head()

,ab,aber,aber nicht,abil,abil adapt,abil build,abil commun,abil develop,abil drive,abil drive requir,abil drive requir role,abil drive requir role would,abil grow,abil grow higher,abil grow higher role,abil grow higher role futur,abil lead,abil manag,abil travel,abil understand,abil work,abil work home,abil work home competit,abil work home competit pay,abil work independ,abl,abl demonstr,abl develop,abl influenc,abl influenc nh,abl influenc nh staff,abl influenc nh staff highest,abl work,abpi,abpi code,abpi code practic,abpi qualifi,abschluss,abteilung,abteilungen,...,year experi within,year experi work,year experi write,year experi write regulatori,year experi write regulatori document,year gap,year gap month,year gap month explain,year gap month explain ck,year medic,year medic write,year regulatori,year relev,year relev experi,year year,york,yorkshir,young,zeit,ziel,zone,zu,zu arbeiten,zu bewerben,zu den,zu entwickeln,zu erfahren,zu erreichen,zu gew,zu gew hrleisten,zu haus,zu sein,zu unterst,zu unterst tzen,zu werden,zum,zur,zusammen,zusammenarbeit,zusammenarbeit mit
0,0.0,0.0,0.0,0.035243,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037658,0.0,0.069103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.029853,0.0,0.0,0.068834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
#Applying MultinomialNaive Bayes classifier to get the job type
classifier=MultinomialNB(alpha=0.6, fit_prior=True, class_prior=None)
classifier.fit(X_train, y_train)
pred = classifier.predict(M_test)


In [37]:
pred.shape

(11771,)

In [38]:
#printing the predicted results
pred[0:100]

array([4, 5, 4, 5, 4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 4, 4, 0, 4, 4, 5, 5, 5,
       5, 4, 4, 4, 4, 5, 4, 5, 4, 4, 4, 5, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 5, 4, 4, 4, 4, 0, 4, 4, 5, 5, 4, 4, 4, 4, 4, 5, 4, 4, 5, 4, 4,
       4, 4, 0, 5, 4, 4, 4, 5, 4, 4, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0,
       4, 4, 4, 5, 4, 4, 5, 4, 4, 0, 4, 5])

In [39]:
#inverting the encoding to get the result in understandable form
pred_type = enc.inverse_transform(pred)
print(pred_type[0:100])

['Permanent' 'Temporary/Seasonal' 'Permanent' 'Temporary/Seasonal'
 'Permanent' 'Permanent' 'Permanent' 'Permanent' 'Permanent'
 'Temporary/Seasonal' 'Permanent' 'Permanent' 'Permanent' 'Permanent'
 'Permanent' 'Permanent' 'Any' 'Permanent' 'Permanent'
 'Temporary/Seasonal' 'Temporary/Seasonal' 'Temporary/Seasonal'
 'Temporary/Seasonal' 'Permanent' 'Permanent' 'Permanent' 'Permanent'
 'Temporary/Seasonal' 'Permanent' 'Temporary/Seasonal' 'Permanent'
 'Permanent' 'Permanent' 'Temporary/Seasonal' 'Permanent'
 'Temporary/Seasonal' 'Permanent' 'Permanent' 'Permanent' 'Permanent'
 'Permanent' 'Permanent' 'Permanent' 'Permanent' 'Permanent'
 'Temporary/Seasonal' 'Permanent' 'Permanent' 'Permanent' 'Permanent'
 'Any' 'Permanent' 'Permanent' 'Temporary/Seasonal' 'Temporary/Seasonal'
 'Permanent' 'Permanent' 'Permanent' 'Permanent' 'Permanent'
 'Temporary/Seasonal' 'Permanent' 'Permanent' 'Temporary/Seasonal'
 'Permanent' 'Permanent' 'Permanent' 'Permanent' 'Any'
 'Temporary/Seasonal' 'Permanen

In [40]:
#making category as the dependent variable for predicting it.
z=train['category']
z

0        Pharmaceutical, Healthcare and Medical Sales
1                                   Clinical Research
2                            Pharmaceutical Marketing
3        Pharmaceutical, Healthcare and Medical Sales
4                          Manufacturing & Operations
                             ...                     
17652                      Manufacturing & Operations
17653                                         Science
17654       Medical Information and Pharmacovigilance
17655                               Clinical Research
17656                               Clinical Research
Name: category, Length: 17657, dtype: object

In [41]:
#applyng Label encoder again to dependent variable to feed it in to the model.
c=LabelEncoder()
z=c.fit_transform(z)

In [42]:
#making a copy of dependent variable
z_train=z.copy()
z_train

array([6, 0, 5, ..., 4, 0, 0])

In [43]:
#Applying MultinomialNaive Bayes classifier to get the job type
classifier=MultinomialNB(alpha=0.7, fit_prior=True, class_prior=None)
classifier.fit(X_train, z_train)
z_pred = classifier.predict(M_test)

In [44]:
#printing the predicted variables
z_pred[0:100]

array([10,  7,  7,  7, 10,  3,  4,  9,  3,  7,  9, 10, 10,  2,  7,  9,  9,
        3,  0,  7,  7,  8,  7, 10,  0,  7, 10,  7, 10,  7, 10,  9,  4,  7,
       10,  7,  4, 10, 10,  4,  0,  4, 10,  9,  0,  7, 10,  0,  9, 10,  9,
        3,  9,  3,  7, 10,  7, 10,  0,  7,  7,  3,  8,  9,  0, 10, 10, 10,
        7,  7, 10,  9, 10, 10, 10, 10,  7, 10, 10,  0,  7, 10, 10, 10,  0,
       10, 10,  9, 10, 10, 10,  7, 10, 10,  7,  3,  4,  7,  9,  9])

In [45]:
#inverting the encoding to get the result in understandable form
pred_category = c.inverse_transform(z_pred)
print(pred_category[0:100])

['Science' 'Pharmacy' 'Pharmacy' 'Pharmacy' 'Science'
 'Medical Affairs / Pharmaceutical Physician'
 'Medical Information and Pharmacovigilance' 'Regulatory Affairs'
 'Medical Affairs / Pharmaceutical Physician' 'Pharmacy'
 'Regulatory Affairs' 'Science' 'Science' 'Manufacturing & Operations'
 'Pharmacy' 'Regulatory Affairs' 'Regulatory Affairs'
 'Medical Affairs / Pharmaceutical Physician' 'Clinical Research'
 'Pharmacy' 'Pharmacy' 'Quality-assurance' 'Pharmacy' 'Science'
 'Clinical Research' 'Pharmacy' 'Science' 'Pharmacy' 'Science' 'Pharmacy'
 'Science' 'Regulatory Affairs'
 'Medical Information and Pharmacovigilance' 'Pharmacy' 'Science'
 'Pharmacy' 'Medical Information and Pharmacovigilance' 'Science'
 'Science' 'Medical Information and Pharmacovigilance' 'Clinical Research'
 'Medical Information and Pharmacovigilance' 'Science'
 'Regulatory Affairs' 'Clinical Research' 'Pharmacy' 'Science'
 'Clinical Research' 'Regulatory Affairs' 'Science' 'Regulatory Affairs'
 'Medical Affairs 

In [46]:
pharma_test.shape

(11771, 2)

In [47]:
#pred_type array for job type
#pred_category for job category 
pharma_test['job_type'] = pred_type
pharma_test['category'] = pred_category

pharma_test.drop('job_description', axis = 'columns', inplace = True)
pharma_test

,job_no,job_type,category
0,Id-1,Permanent,Science
1,Id-2,Temporary/Seasonal,Pharmacy
2,Id-4,Permanent,Pharmacy
3,Id-6,Temporary/Seasonal,Pharmacy
4,Id-7,Permanent,Science
...,...,...,...
11766,Id-29421,Permanent,Quality-assurance
11767,Id-29422,Permanent,Medical Information and Pharmacovigilance
11768,Id-29425,Permanent,Medical Information and Pharmacovigilance
11769,Id-29426,Permanent,Regulatory Affairs


In [48]:
print(pharma_test.job_type.unique())
print(pharma_test.category.unique())

['Permanent' 'Temporary/Seasonal' 'Any' 'Part-Time' 'Contract/Interim']
['Science' 'Pharmacy' 'Medical Affairs / Pharmaceutical Physician'
 'Medical Information and Pharmacovigilance' 'Regulatory Affairs'
 'Manufacturing & Operations' 'Clinical Research' 'Quality-assurance'
 'Pharmaceutical, Healthcare and Medical Sales'
 'Data Management and Statistics' 'Pharmaceutical Marketing']


In [49]:
print(pharma_test.job_type.value_counts())
print(pharma_test.category.value_counts())

Permanent             8446
Temporary/Seasonal    3080
Any                    190
Part-Time               53
Contract/Interim         2
Name: job_type, dtype: int64
Science                                         3211
Medical Information and Pharmacovigilance       2150
Pharmacy                                        1802
Quality-assurance                               1533
Regulatory Affairs                              1467
Medical Affairs / Pharmaceutical Physician       589
Clinical Research                                548
Manufacturing & Operations                       313
Pharmaceutical Marketing                          86
Pharmaceutical, Healthcare and Medical Sales      36
Data Management and Statistics                    36
Name: category, dtype: int64


In [50]:
submission = pharma_test.to_csv('TFidfsubmission2.csv')

In [51]:
from google.colab import files
files.download("TFidfsubmission2.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>